<a href="https://colab.research.google.com/github/0xD4rky/DL_101/blob/main/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import numpy as np
import pandas as pd

In [50]:
df=pd.read_csv('/content/diabetes.csv')

In [51]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [52]:
df.corr()['Outcome']
#I want to check how much each column impacts the outcome

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [53]:
X=df.iloc[:,:-1].values
# My X will be all the columns excluding the last one

y=df.iloc[:,-1].values
# : represents all the rows
# :-1 represents all the columns except the last one

Scaling the data

In [54]:
# Now we will scale the data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# creating obj of the class

In [55]:
X = scaler.fit_transform(X)
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [56]:
X.shape

(768, 8)

In [57]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [108]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [59]:
model= Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
# we chose sigmoid as activation as it is a classification model

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [60]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 11ms/step - loss: 0.7468 - accuracy: 0.4739 - val_loss: 0.7268 - val_accuracy: 0.4935
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6743 - accuracy: 0.5847 - val_loss: 0.6626 - val_accuracy: 0.5844
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6215 - accuracy: 0.6612 - val_loss: 0.6169 - val_accuracy: 0.6688
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5841 - accuracy: 0.7085 - val_loss: 0.5829 - val_accuracy: 0.7403
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5572 - accuracy: 0.7410 - val_loss: 0.5594 - val_accuracy: 0.7532
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5358 - accuracy: 0.7410 - val_loss: 0.5410 - val_accuracy: 0.7597
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5196 - accuracy: 0.7459 - val_loss: 0.5266 - val_accuracy: 0.7532
Epoch 8/100


HYPERPARAMETER TUNING

In [61]:
 !pip install -U keras-tuner
 # downloading the main tuning library of keras

In [62]:
import kerastuner as kt

In [63]:
# 1. Adding an optimizer

# we are creating an object
def build_model(hp):
  # hp= hyperparameter

  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  opt=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  # we are giving a choice of optimizers
  model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [64]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',
                      max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [65]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [66]:
tuner.get_best_hyperparameters()[0].values
# finding the best hyperpara

{'optimizer': 'rmsprop'}

In [67]:
model=tuner.get_best_models(num_models=1)[0]

# Since we passed 4 optimizer options,
# we will have 4 models. So out of those
# 4 models we are displaying the best model

In [68]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [69]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,
          validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5032 - accuracy: 0.7492 - val_loss: 0.5032 - val_accuracy: 0.7922
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4924 - accuracy: 0.7573 - val_loss: 0.4943 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4832 - accuracy: 0.7622 - val_loss: 0.4887 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4769 - accuracy: 0.7606 - val_loss: 0.4842 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4719 - accuracy: 0.7638 - val_loss: 0.4818 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4682 - accuracy: 0.7671 - val_loss: 0.4802 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4647 - accuracy: 0.7687 - val_loss: 0.4784 - val_accuracy: 0.8052
Epoch 14

Finding the right number of neurons

In [70]:
def build_model(hp):
  model=Sequential()
  units= hp.Int('units',min_value=8,max_value=128,step=8)
  # It will build multiple models starting from 8 neurons
  # And it will increase by a count of 8 in each new model
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [71]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy'
                        ,max_trials=5
                        ,directory='mydir')

# creating an object of the build_model class

Reloading Tuner from mydir/untitled_project/tuner0.json


In [72]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [73]:
tuner.get_best_hyperparameters()[0].values

{'units': 72}

In [74]:
 model=tuner.get_best_models(num_models=1)[0]

In [75]:
model.fit(X_train,y_train,epochs=100,batch_size=32,initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 1s 2ms/step - loss: 0.5370 - accuracy: 0.7280
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5106 - accuracy: 0.7427
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4944 - accuracy: 0.7557
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4821 - accuracy: 0.7655
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4732 - accuracy: 0.7687
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4679 - accuracy: 0.7704
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4633 - accuracy: 0.7752
Epoch 14/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4596 - accuracy: 0.7752
Epoch 15/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4578 - accuracy: 0.7769
Epoch 16/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4551 - accuracy: 0.7785
Epo

Tuning number of Hidden Layers

In [88]:
def build_models(hp):
  model=Sequential()

  model.add(Dense(72,activation='relu',input_dim=8))
  #we add a layer before to reduce complexity regarding input dimernsion

  for i in range(hp.Int('num_layers'),min_value=1,max_value=10):

    model.add(Dense(72,activation='relu'))


  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='adam',loss='bianry_crossentropy',metrics=['accuracy'])

  return model


In [89]:
tunerr=kt.RandomSearch(build_models,
                      objective='val_accuracy',
                      max_trials=3,
                      directory='mydir')

Reloading Tuner from mydir/untitled_project/tuner0.json


In [90]:
tunerr.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [87]:
tunerr.get_best_hyperparameters()[0].values

{'units': 72}

Tuning no of layers together with neurons


In [113]:
def build_model(hp):
  model=Sequential()
  counter=0
  #units=hp.Int('')
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter==0:
      model.add(
          Dense(
                hp.Int('units'+ str(i),min_value=8,max_value=128,step=8),
                activation=hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid']), #we have to change the string name in each loop acc to keras
                input_dim=8
                )
          )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]))) #we have to change the string name in each loop acc to keras
    else:
      model.add(
        Dense(
              hp.Int('units'+ str(i),min_value=8,max_value=128,step=8),
              activation=hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid'])
              )
        )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]))) #we have to change the string name in each loop acc to keras
    counter+=1
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['adam','rmsprop','sgd','nadam','adadelta']),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
  return model


In [114]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=3,
                      directory='mydir',
                      project_name='final1')

In [115]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.6168830990791321

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 09s


In [116]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units0': 88,
 'activation0': 'sigmoid',
 'dropout0': 0.2,
 'optimizer': 'sgd',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1}

In [117]:
model=tuner.get_best_models(num_models=1)[0]

In [118]:
model.fit(X_train,y_train,epochs=200,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/200
20/20 [==============================] - 1s 14ms/step - loss: 0.6779 - accuracy: 0.6303 - val_loss: 0.6766 - val_accuracy: 0.6429
Epoch 8/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6698 - accuracy: 0.6482 - val_loss: 0.6718 - val_accuracy: 0.6429
Epoch 9/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6695 - accuracy: 0.6515 - val_loss: 0.6679 - val_accuracy: 0.6429
Epoch 10/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6668 - accuracy: 0.6531 - val_loss: 0.6651 - val_accuracy: 0.6429
Epoch 11/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6652 - accuracy: 0.6531 - val_loss: 0.6632 - val_accuracy: 0.6429
Epoch 12/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6600 - accuracy: 0.6531 - val_loss: 0.6607 - val_accuracy: 0.6429
Epoch 13/200
20/20 [==============================] - 0s 4ms/step - loss: 0.6598 - accuracy: 0.6531 - val_loss: 0.6588 - val_accuracy: 0.6429
Epoch 14